Array Addition, subtraction, Multiplication

In [ ]:
%%writefile 1.cu

#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>

#define n 10

__global__ void array_add(int *a, int *b, int *c){
  int tid = threadIdx.x + blockIdx.x * blockDim.x;

  if(tid < n){
    for(int i=0; i<n; i++){
      c[i] = a[i] + b[i];   //+, -, *
    }
    //result = c[i];
  }
}

int main(){
  int a[n], b[n], c[n];

  for(int i=0; i<n; i++){
    a[i]=i+1;
    b[i]=i+1;
  }

  int *d_a, *d_b, *d_c;

  cudaMalloc((void**)&d_a, n*sizeof(int));
  cudaMalloc((void**)&d_b, n*sizeof(int));
  cudaMalloc((void**)&d_c, n*sizeof(int));

  cudaMemcpy(d_a, a, n*sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, b, n*sizeof(int), cudaMemcpyHostToDevice);

  int threadsPerBlock = 256;
  int blocksPerGrid = (n + threadsPerBlock -1) / threadsPerBlock;

  array_add<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c);

  cudaMemcpy(c, d_c, n*sizeof(int), cudaMemcpyDeviceToHost);

  for(int i=0; i<n; i++){
    printf("%d ", c[i]);
  }
  printf("\n");

  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);

  return 0;
}


Overwriting 1.cu


In [ ]:
!nvcc -o 1 1.cu

In [ ]:
!./1

0 0 0 0 0 0 0 0 0 0 


In [ ]:
%%writefile 2.cu

#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>

#define n 10

__global__ void arrayMax(int *a, int *b, int max, int min){
  int tid = threadIdx.x + blockIdx.x * blockDim.x;

  if(tid < n){
    max = 0;
    for(int i=0; i<n; i++){
      if (a[i]>max){
        max = a[i];
      }
    }
    min =0;
    for(int i=0; i<n;i++){
      if(a[i]<=min){
        min = a[i];
    }
    }
  }
}

int main(){
  int a[n], b[n], max, min;

  for(int i=0; i<n; i++){
    a[i]=i+1;
    b[i]=i+1;
  }

  int *d_a, *d_b, d_max, d_min;

  cudaMalloc((void**)&d_a, n*sizeof(int));
  cudaMalloc((void**)&d_b, n*sizeof(int));
  cudaMalloc((void**)&d_max, sizeof(int));
  cudaMalloc((void**)&d_min, sizeof(int));

  cudaMemcpy(d_a, a, n*sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, b, n*sizeof(int), cudaMemcpyHostToDevice);

  int threadsPerBlock = 256;
  int blocksPerGrid = (n+threadsPerBlock-1)/threadsPerBlock;

  arrayMax<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_max, d_min);

  cudaMemcpy(&max, &d_max, sizeof(int), cudaMemcpyDeviceToHost);
  cudaMemcpy(&min, &d_min, sizeof(int), cudaMemcpyDeviceToHost);

  printf("%d\n",max);
  printf("%d\n",max);

  cudaFree(d_a);
  cudaFree(d_b);
  //cudaFree(d_max);

  return 0;

}

Overwriting 2.cu


In [ ]:
!nvcc -o 2 2.cu

In [ ]:
!./2

1
1


In [ ]:
%%writefile 3.cu

#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>

#define n 10

__global__ void arrayMax(int *a, int *b, int *max, int min){
  int tid = threadIdx.x + blockIdx.x * blockDim.x;

  if(tid < n){
    atomicMax(max, a[tid]);
  }
}


int main(){
  int a[n], b[n], max, min;

  for(int i=0; i<n; i++){
    a[i]=i+1;
    b[i]=i+1;
  }

  int *d_a, *d_b, *d_max, d_min;

  cudaMalloc((void**)&d_a, n*sizeof(int));
  cudaMalloc((void**)&d_b, n*sizeof(int));
  cudaMalloc((void**)&d_max, sizeof(int));
  cudaMalloc((void**)&d_min, sizeof(int));

  cudaMemcpy(d_a, a, n*sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, b, n*sizeof(int), cudaMemcpyHostToDevice);

  int threadsPerBlock = 256;
  int blocksPerGrid = (n+threadsPerBlock-1)/threadsPerBlock;

  arrayMax<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_max, d_min);

  cudaMemcpy(&max, d_max, sizeof(int), cudaMemcpyDeviceToHost);
  cudaMemcpy(&min, &d_min, sizeof(int), cudaMemcpyDeviceToHost);

  printf("%d\n",max);
  printf("%d\n",max);

  cudaFree(d_a);
  cudaFree(d_b);
  //cudaFree(d_max);

  return 0;

}

Overwriting 3.cu


In [ ]:
!nvcc -o 3 3.cu

In [ ]:
!./3

10
10


In [ ]:
%%writefile 4.cu

#include <stdio.h>
#include <cuda_runtime.h>

#define N 10

__device__ int d_a[N];
__device__ int d_b[N];
__device__ int d_c[N];

__global__ void add_array(){
  int tid = threadIdx.x + blockIdx.x * blockDim.x;

  if(tid < N){
    d_c[tid] = d_a[tid] + d_b[tid];
  }
}

void initializeArrayA(int *a){
  cudaMemcpyToSymbol(d_a, a, N*sizeof(int), 0, cudaMemcpyHostToDevice);
}

void initArrayB(int *b){
  cudaMemcpyToSymbol(d_b, b, N*sizeof(int), 0, cudaMemcpyHostToDevice);
}

void retrieveResFromC(int *c){
  cudaMemcpyFromSymbol(c, d_c, N*sizeof(int), 0, cudaMemcpyDeviceToHost);
}

int main(){
  int a[N], b[N], c[N];

  for(int i=0; i< N; i++){
    a[i]=i+1;
    b[i]=i+2;
  }

  initializeArrayA(a);
  initArrayB(b);

  int threadsPerBlock = 256;
  int blocksPerGrid = (N + threadsPerBlock -1)/threadsPerBlock;

  add_array<<<blocksPerGrid, threadsPerBlock>>>();
  cudaDeviceSynchronize();

  retrieveResFromC(c);

  for(int i=0; i<N; i++){
    printf("%d ", c[i]);
  }

  return 0;

}


Overwriting 4.cu


In [ ]:
!nvcc -o 4 4.cu

In [ ]:
!./4

3 5 7 9 11 13 15 17 19 21 

In [ ]:
%%writefile 5.cu

#include <stdio.h>
#include <cuda.h>

#define N 100

__global__ void primeNum(){
  int id = threadIdx.x + blockIdx.x * blockDim.x;

  if(id < N){
    int count = 0;
    for(int j=0; j<=id; j++){
      if(id%j==0){
        count++;
      }
    }
    if(count == 2){
      printf("%d is prime\n", id);
    }else{
      printf("%d is not prime\n", id);
    }
  }
}

int main(){
  int threadsPerBlock = 256;
  int blocksPerGrid = (N + threadsPerBlock-1)/threadsPerBlock;

  primeNum<<<blocksPerGrid, threadsPerBlock>>>();
  cudaDeviceSynchronize();

  return 0;
}

Overwriting 5.cu


In [ ]:
!nvcc -o 5 5.cu

In [ ]:
!./5

97 is prime
37 is prime
41 is prime
43 is prime
47 is prime
53 is prime
59 is prime
61 is prime
2 is prime
3 is prime
5 is prime
7 is prime
11 is prime
13 is prime
17 is prime
19 is prime
23 is prime
29 is prime
31 is prime
67 is prime
71 is prime
73 is prime
79 is prime
83 is prime
89 is prime
96 is not prime
98 is not prime
99 is not prime
0 is not prime
1 is not prime
4 is not prime
6 is not prime
8 is not prime
9 is not prime
10 is not prime
12 is not prime
14 is not prime
15 is not prime
16 is not prime
18 is not prime
20 is not prime
21 is not prime
22 is not prime
24 is not prime
25 is not prime
26 is not prime
27 is not prime
28 is not prime
30 is not prime
32 is not prime
33 is not prime
34 is not prime
35 is not prime
36 is not prime
38 is not prime
39 is not prime
40 is not prime
42 is not prime
44 is not prime
45 is not prime
46 is not prime
48 is not prime
49 is not prime
50 is not prime
51 is not prime
52 is not prime
54 is not prime
55 is not prime
56 is not prime
57 is 

In [ ]:
%%writefile 6.cu

#include <stdio.h>
#define N 15

__global__ void sumWithSharedMem(int *d_input, int *d_output, int n) {
    extern __shared__ int sdata[]; // Dynamically allocated shared memory

    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Load data into shared memory
    if (idx < n) {
        sdata[threadIdx.x] = d_input[idx];
    } else {
        sdata[threadIdx.x] = 0;
    }

    __syncthreads();

    // Perform block-wise sum
    if (threadIdx.x == 0) {
        int blockSum = 0;
        for (int i = 0; i < blockDim.x; i++) { // Corrected condition
            blockSum += sdata[i];
        }
        d_output[blockIdx.x] = blockSum;
    }
}

int main() {
    int h_input[N], *d_input, *d_output;
    int threadsPerBlock = 4;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock; // Dynamic block calculation
    int h_output[blocksPerGrid]; // Adjust output array size dynamically
    int finalSum = 0; // Initialize finalSum

    for (int i = 0; i < N; i++) {
        h_input[i] = i + 1;
    }

    cudaMalloc((void **)&d_input, N * sizeof(int));
    cudaMalloc((void **)&d_output, blocksPerGrid * sizeof(int));

    cudaMemcpy(d_input, h_input, N * sizeof(int), cudaMemcpyHostToDevice);

    sumWithSharedMem<<<blocksPerGrid, threadsPerBlock, threadsPerBlock * sizeof(int)>>>(d_input, d_output, N);

    cudaMemcpy(h_output, d_output, blocksPerGrid * sizeof(int), cudaMemcpyDeviceToHost);

    for (int i = 0; i < blocksPerGrid; i++) {
        finalSum += h_output[i];
    }

    printf("Final Sum: %d\n", finalSum);

    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}


Overwriting 6.cu


In [ ]:
!nvcc -o 6 6.cu

In [ ]:
!./6

Final Sum: 0


In [1]:
%%writefile 7.cu

#include <stdio.h>
#define N 15

__global__ void sumWithSharedMem(int *d_input, int *d_output, int n){
  extern __shared__ int Sdata[];

  int tid = threadIdx.x + blockIdx.x * blockDim.x;

  if(tid < n){
    Sdata[threadIdx.x] = d_input[tid];
  }
  else{
    Sdata[threadIdx.x] = 0;
  }
  __syncthreads();

  if(threadIdx.x == 0){
    int blockSum =0;
    for(int i=0; i<blockDim.x; i++){
      blockSum += Sdata[i];
    }
    d_output[blockIdx.x] = blockSum;
  }
}

int main(){
  int h_input[N];
  int threadsPerBlock = 4;
  int blocksPerGrid = (N+threadsPerBlock-1)/threadsPerBlock;
  int h_output[blocksPerGrid];

  for(int i=0; i<N; i++){
    h_input[i] = i+1;
  }

  int *d_input, *d_output;
  cudaMalloc((void**)&d_input, N*sizeof(int));
  cudaMalloc((void**)&d_output, blocksPerGrid*sizeof(int));

  cudaMemcpy(d_input, h_input, N*sizeof(int), cudaMemcpyHostToDevice);

  sumWithSharedMem<<<blocksPerGrid, threadsPerBlock, threadsPerBlock*sizeof(int)>>>(d_input, d_output, N);

  cudaMemcpy(h_output, d_output, blocksPerGrid*sizeof(int), cudaMemcpyDeviceToHost);

  int finalSum =0;
  for(int i=0; i<blocksPerGrid; i++){
    finalSum += h_output[i];
  }

  printf("%d \n", finalSum);

  cudaFree(d_input);
  cudaFree(d_output);
  return 0;
}

Writing 7.cu


In [2]:
!nvcc -o 7 7.cu

In [3]:
!./7

120 


In [7]:
%%writefile 8.cu

#include <stdio.h>
#define N 16

__device__ int blockReduction(int *Sdata, int blocksize){
  int blockSum = 0;
  for(int i=0; i<blocksize; i++){
    blockSum += Sdata[i];
  }
  return blockSum;
}

__global__ void SharedSum(int *d_input, int *d_output, int n){
  extern __shared__ int Sdata[];

  int tid = threadIdx.x + blockIdx.x * blockDim.x;

  if(tid < n){
    Sdata[threadIdx.x] = d_input[tid];
  }
  else{
    Sdata[threadIdx.x] = 0;
  }
  __syncthreads();

  if(threadIdx.x == 0){
    int blockSum = blockReduction(Sdata, blockDim.x);
    d_output[blockIdx.x] = blockSum;
  }
}

int main(){
  int h_input[N];
  int threadsPerBlock = 4;
  int blocksPerGrid = (N + threadsPerBlock-1)/threadsPerBlock;
  int h_output[blocksPerGrid];

  for(int i=0; i<N; i++){
    h_input[i] = i+1;
  }

  int *d_input, *d_output;

  cudaMalloc((void**)&d_input, N*sizeof(int));
  cudaMalloc((void**)&d_output, blocksPerGrid*sizeof(int));

  cudaMemcpy(d_input, h_input, N*sizeof(int), cudaMemcpyHostToDevice);

  SharedSum<<<blocksPerGrid, threadsPerBlock, threadsPerBlock*sizeof(int)>>>(d_input, d_output, N);

  cudaMemcpy(h_output, d_output, blocksPerGrid*sizeof(int), cudaMemcpyDeviceToHost);

  int finalSum = 0;
  for(int i=0; i<blocksPerGrid; i++){
    finalSum += h_output[i];
  }

  printf("final sum: %d\n", finalSum);

  cudaFree(d_input);
  cudaFree(d_output);

  return 0;
}

Overwriting 8.cu


In [8]:
!nvcc -o 8 8.cu

In [9]:
!./8

final sum: 136


In [11]:
%%writefile 9.cu

#include <stdio.h>
#define N 1024

__global__ void AtomicAdd(int *d_input, int *d_output){
  int tid = threadIdx.x;

  if(tid<N){
    atomicAdd(d_output, d_input[tid]);
  }
}

int main(){
  int h_input[N], h_output = 0;
  for(int i=0; i<N; i++){
    h_input[i] = i+1;
  }

  int *d_input, *d_output;
  cudaMalloc((void**)&d_input, N*sizeof(int));
  cudaMalloc((void**)&d_output, sizeof(int));

  cudaMemcpy(d_input, h_input, N*sizeof(int), cudaMemcpyHostToDevice);
  AtomicAdd<<<1, 1024>>>(d_input, d_output);
  cudaMemcpy(&h_output, d_output, sizeof(int), cudaMemcpyDeviceToHost);

  printf("%d ", h_output);

  cudaFree(d_input);
  cudaFree(d_output);

  return 0;
}

Overwriting 9.cu


In [12]:
!nvcc -o 9 9.cu

In [13]:
!./9

524800 

In [8]:
%%writefile 10.cu

#include <stdio.h>
#define N 1024

__global__ void AtomicAddMaxMin(int *d_input, int *d_sum, int *d_max, int *d_min){
  int tid = threadIdx.x + blockIdx.x * blockDim.x;

  if(tid < N){
    atomicAdd(d_sum, d_input[tid]);
    atomicMax(d_max, d_input[tid]);
    atomicMin(d_min, d_input[tid]);
  }
}

int main(){
  int h_input[N];
  int h_sum, h_max, h_min = 0;
  for(int i=0; i<N; i++){
    h_input[i] = i+1;
  }

  int threadsPerBlock = 256;
  int blocksPerGrid = (N+threadsPerBlock-1)/threadsPerBlock;

  int *d_input, *d_sum, *d_max, *d_min;
  cudaMalloc((void**)&d_input, N*sizeof(int));
  cudaMalloc((void**)&d_sum, sizeof(int));
  cudaMalloc((void**)&d_max, sizeof(int));
  cudaMalloc((void**)&d_min, sizeof(int));

  cudaMemcpy(d_input, h_input, N*sizeof(int), cudaMemcpyHostToDevice);
  AtomicAddMaxMin<<<blocksPerGrid, threadsPerBlock>>>(d_input, d_sum, d_max, d_min);
  cudaMemcpy(&h_sum, d_sum, sizeof(int), cudaMemcpyDeviceToHost);
  cudaMemcpy(&h_min, d_min, sizeof(int), cudaMemcpyDeviceToHost);
  cudaMemcpy(&h_max, d_max, sizeof(int), cudaMemcpyDeviceToHost);

  printf("sum : %d\n max: %d\n min: %d\n", h_sum, h_max, h_min);

  return 0;

}

Overwriting 10.cu


In [9]:
!nvcc -o 10 10.cu

In [10]:
!./10

sum : 524800
 max: 1024
 min: 0


In [1]:
%%writefile 11.cu

#include <stdio.h>
#define N 5

__global__ void matrix_addition(int *a, int *b, int *c, int n){
  int tid = threadIdx.x + blockIdx.x * blockDim.x;

  if(tid < n*n){
    int i = tid / n;
    int j = tid % n;
    c[i * n + j] = a[i * n + j]+ b[i*n+j];
  }
}

int main(){
  int a[N][N], b[N][N], c[N][N];
  for(int i=0; i<N; i++){
    for(int j=0; j<N; j++){
      a[i][j] = i+1;
      b[i][j] = i+1;
    }
  }

  int threadsPerBlock = N;
  int blocksPerGrid = (N*N+threadsPerBlock-1)/threadsPerBlock;

  int *d_a, *d_b, *d_c;
  cudaMalloc((void**)&d_a, N*N*sizeof(int));
  cudaMalloc((void**)&d_b, N*N*sizeof(int));
  cudaMalloc((void**)&d_c, N*N*sizeof(int));

  cudaMemcpy(d_a, a, N*N*sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, b, N*N*sizeof(int), cudaMemcpyHostToDevice);

  matrix_addition<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, N);

  cudaMemcpy(c, d_c, N*N*sizeof(int), cudaMemcpyDeviceToHost);

  for(int i=0; i<N; i++){
    for(int j=0; j<N; j++){
      printf("%d ", c[i][j]);
    }
    printf("\n");
  }

  return 0;
}

Writing 11.cu


In [2]:
!nvcc -o 11 11.cu

In [3]:
!./11

2 2 2 2 2 
4 4 4 4 4 
6 6 6 6 6 
8 8 8 8 8 
10 10 10 10 10 


In [7]:
%%writefile 12.cu

#include <stdio.h>
#include <stdlib.h>

__global__ void matrix_add(int *a, int *b, int *c, int n){
  int tid = threadIdx.x + blockIdx.x * blockDim.x;

  if(tid < n * n){
    int i = tid / n;
    int j = tid % n;
    c[i*n+j] = a[i*n+j] + b[i*n+j];
  }
}

int main(){
  int sizes[] = {5,10,15};
  int n;


  for(int i=0; i<3; i++){
    n = sizes[i];
    int size = n*n*sizeof(int);

    int *a, *b, *c;
    a = (int*)malloc(size);
    b = (int*)malloc(size);
    c = (int*)malloc(size);

    for(int i=0; i<n*n; i++){
      a[i] = i+1;
      b[i] = i+1;
    }

    int *d_a, *d_b, *d_c;
    cudaMalloc((void**)&d_a, size);
    cudaMalloc((void**)&d_b, size);
    cudaMalloc((void**)&d_c, size);

    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocksPerGrid = (n*n + threadsPerBlock-1)/threadsPerBlock;

    matrix_add<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);

    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    for(int i=0; i<n; i++){
      for(int j=0; j<n; j++){
        printf("%d ", c[i*n+j]);
      }
      printf("\n");
    }
    printf("\n");
  }

  return 0;
}

Overwriting 12.cu


In [8]:
!nvcc -o 12 12.cu

In [9]:
!./12

2 4 6 8 10 
12 14 16 18 20 
22 24 26 28 30 
32 34 36 38 40 
42 44 46 48 50 

2 4 6 8 10 12 14 16 18 20 
22 24 26 28 30 32 34 36 38 40 
42 44 46 48 50 52 54 56 58 60 
62 64 66 68 70 72 74 76 78 80 
82 84 86 88 90 92 94 96 98 100 
102 104 106 108 110 112 114 116 118 120 
122 124 126 128 130 132 134 136 138 140 
142 144 146 148 150 152 154 156 158 160 
162 164 166 168 170 172 174 176 178 180 
182 184 186 188 190 192 194 196 198 200 

2 4 6 8 10 12 14 16 18 20 22 24 26 28 30 
32 34 36 38 40 42 44 46 48 50 52 54 56 58 60 
62 64 66 68 70 72 74 76 78 80 82 84 86 88 90 
92 94 96 98 100 102 104 106 108 110 112 114 116 118 120 
122 124 126 128 130 132 134 136 138 140 142 144 146 148 150 
152 154 156 158 160 162 164 166 168 170 172 174 176 178 180 
182 184 186 188 190 192 194 196 198 200 202 204 206 208 210 
212 214 216 218 220 222 224 226 228 230 232 234 236 238 240 
242 244 246 248 250 252 254 256 258 260 262 264 266 268 270 
272 274 276 278 280 282 284 286 288 290 292 294 296 298 300 
302 304 

In [10]:
%%writefile 13.cu

#include <stdio.h>
#define N 5

__global__ void matrix_mult(int *a, int *b, int *c, int n){
  int tid = threadIdx.x + blockIdx.x * blockDim.x;

  if(tid < n*n){
    int row = tid / n;
    int col = tid % n;

    int sum =0;
    for(int i=0; i<n;i++){
      sum += a[row*n+i] * b[i*n+col];
    }
    c[row*n+col] = sum;
  }
}

int main(){
  int a[N][N] = {0};
  int b[N][N] = {0};
  int c[N][N] = {0};
  int size = N*N*sizeof(int);

  for(int i=0; i<N; i++){
    for(int j=0; j<N; j++){
      a[i][j] = rand() % 10;
      b[i][j] = rand() % 10;
    }
  }

  int *d_a, *d_b, *d_c;
  cudaMalloc((void**)&d_a, size);
  cudaMalloc((void**)&d_b, size);
  cudaMalloc((void**)&d_c, size);

  cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

  int threadsPerBlock = 256;
  int blocksPerGrid = (N*N + threadsPerBlock -1)/threadsPerBlock;

  matrix_mult<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, N);

  cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

  for(int i=0; i<N; i++){
    for(int j=0; j<N;j++){
      printf("%d ", c[i][j]);
    }
    printf("\n");
  }
  printf("\n");

  return 0;

}

Overwriting 13.cu


In [11]:
!nvcc -o 13 13.cu

In [12]:
!./13

130 234 162 129 72 
42 55 24 21 17 
96 165 110 90 57 
149 177 85 79 72 
124 199 142 104 51 



In [8]:
%%writefile 14.c

#include <stdio.h>
#define N 5

void matrix_mul(int a[N][N], int b[N][N], int c[N][N], int n){
  #pragma acc parallel loop collapse(2) copyin(a[0:N][0:N], b[0:N][0:N]) copyout(c[0:N][0:N])
  for(int row =0; row<n; row++){
    for(int col=0; col<n; col++){
      int sum =0;
      for(int i=0; i<n; i++){
        sum += a[row][i] * b[i][col];
      }
      c[row][col] = sum;
    }
  }
}

int main(){

}